# Телемаркетинг

<b>Данные</b>:

Есть файлы с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки: FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM. И есть файл с логами по подключениям в системе.

Особенности данных:
- сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
- поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
- продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
- если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается

<b>Задача</b>:

Проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе. Сохранить результат в csv файл с разделителем ';', содержащий корректные подключения.

## Загрузка файлов и объединение в один датафрейм

In [1]:
import pandas as pd
import os

import warnings

warnings.filterwarnings('ignore')

In [2]:
# файл с логами
logs = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-n-laschevskaja/lesson_5/prod_activations_logs.csv',\
                   sep=';')

In [3]:
logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43


In [4]:
# путь до папки, где лежат файлы о продажах
path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-n-laschevskaja/lesson_5/'

In [5]:
# список файлов с продажами, начинающимися на 'tm_sales'
list_files = [i for i in os.listdir(path) if i.startswith('tm_sales')]

In [6]:
list_files

['tm_sales_1.csv', 'tm_sales_2.csv', 'tm_sales_3.csv']

In [7]:
# создаем пустой датафрейм с нужными колонками
df = pd.DataFrame(columns=['SUBS_ID', 'PROD_ID', 'FILIAL_ID', 'ACT_DTTM'])

# объединяем файлы в один датафрейм
for file in list_files:
    path_file = path + file # путь до каждого файла
    tm = pd.read_csv(path_file, sep=';') # чтение файла
    df = pd.concat([df, tm]) # объединение файлов в один в пустой датафрейм

In [8]:
# сбросили индексы в датафрейме
df = df.reset_index(drop=True)

In [9]:
df

,SUBS_ID,PROD_ID,FILIAL_ID,ACT_DTTM
0,id4651830,1954,1,20-03-2020 14:59
1,7646509,6431,5,19-03-2020 13:00
2,id7412683,3313,4,22-03-2020 17:25
3,id5416547,1743,3,17-03-2020 10:17
4,NaN,7421,4,12-03-2020 11:25
5,8362218,9879,7,05-03-2020 11:42
6,2185490,3210,2,16-03-2020 16:28
7,id5764122,1499,3,18-03-2020 15:44
8,id7642700,3020,6,15-03-2020 14:21
9,id1374509,5677,2,17-03-2020 11:48


Все файлы по продажам собраны в один датафрейм.

## Предобработка и фильтрация данных в файле о продажах

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SUBS_ID    9 non-null      object
 1   PROD_ID    10 non-null     object
 2   FILIAL_ID  10 non-null     object
 3   ACT_DTTM   10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


В датасете есть пропуск в колонке SUBS_ID. По условию, если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается. Удалим пропуски.

In [11]:
df = df.dropna(subset=['SUBS_ID'])

Колонка с датой и временем (ACT_DTTM) содержит тип данных object, заменим на тип datetime. Воспользуемся аргументом format='%d-%m-%Y %H:%M'

In [12]:
df['ACT_DTTM'] = pd.to_datetime(df.ACT_DTTM, format='%d-%m-%Y %H:%M')

In [13]:
# проверим типы данных
df.dtypes

SUBS_ID              object
PROD_ID              object
FILIAL_ID            object
ACT_DTTM     datetime64[ns]
dtype: object

Значения SUBS_ID не всегда начинаются с "id", добавим, где этого нет.

In [14]:
def add_id(value):
    if not value.startswith('id'): # если значение без id
        return 'id' + value
    else:
        return value

In [15]:
df['SUBS_ID'] = df['SUBS_ID'].apply(add_id)

In [16]:
df

,SUBS_ID,PROD_ID,FILIAL_ID,ACT_DTTM
0,id4651830,1954,1,2020-03-20 14:59:00
1,id7646509,6431,5,2020-03-19 13:00:00
2,id7412683,3313,4,2020-03-22 17:25:00
3,id5416547,1743,3,2020-03-17 10:17:00
5,id8362218,9879,7,2020-03-05 11:42:00
6,id2185490,3210,2,2020-03-16 16:28:00
7,id5764122,1499,3,2020-03-18 15:44:00
8,id7642700,3020,6,2020-03-15 14:21:00
9,id1374509,5677,2,2020-03-17 11:48:00


Сейчас датасет с нужными типами данных, без лишних строк и с правильными значениями в колонке SUBS_ID. Можно объединять данные с файлом логов.

## Предобработка файла с логами

Изменим типы данных в двух колонках на datetime в файле с логами.

In [17]:
logs['START_DTTM'] = pd.to_datetime(logs.START_DTTM, format='%d-%m-%Y %H:%M')
logs['END_DTTM'] = pd.to_datetime(logs.END_DTTM, format='%d-%m-%Y %H:%M')

Продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM). 

Для начала добавим столбец difference, в котором запишем время между началом подключения и окончанием. Затем отберем только те записи, где difference больше 5 минут.

In [18]:
logs['difference'] = logs.END_DTTM - logs.START_DTTM

In [19]:
logs = logs[logs.difference >= '5m']

In [20]:
# сбросили индексы в датафрейме
logs = logs.reset_index(drop=True)

In [21]:
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,255 days 06:35:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
4,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
5,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
6,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,256 days 11:32:00


Файл с логами готов для дальнейшей работы: в нем изменены типы данных и удалены ненужные подключения.

## Объединение файла с продажами и файла с логами

Объединим файлы с товарами и файл с логами по общим колонкам SUBS_ID и PROD_ID, удалив колонку ACT_DTTM - она дублирует колонку START_DTTM.

In [22]:
full_df = df.merge(logs, on=['SUBS_ID', 'PROD_ID']).drop(columns=['ACT_DTTM'], axis=1)

In [23]:
full_df

,SUBS_ID,PROD_ID,FILIAL_ID,START_DTTM,END_DTTM,difference
0,id4651830,1954,1,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id5416547,1743,3,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
2,id2185490,3210,2,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
3,id7642700,3020,6,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [24]:
# отсортируем данные по SUBS_ID
full_df.sort_values('SUBS_ID')

,SUBS_ID,PROD_ID,FILIAL_ID,START_DTTM,END_DTTM,difference
2,id2185490,3210,2,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
0,id4651830,1954,1,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id5416547,1743,3,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id7642700,3020,6,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [25]:
full_df.to_csv('sales.csv')

## Выводы

В ходе данного проекта было сделано:
- объединены файлы по продажам в один датасет
- предобработан и отфильтрован датасет с продажами
- предобработан и отфильтрован файл с логами
- объединены файл по продажам и файл с логами в один датасет только с нужными подключениями
- итоговый датасет сохранен в файл .csv